In [2]:
using Gridap

Define the problem:

$$
  \begin{cases} 
    -\Delta u = f & \text{ in } \Omega\\
    u = g & \text{ in } \partial \Omega
  \end{cases}
$$

Where $\Omega \subset \mathbb{R}^2 \text{ and } \mathbb{R}^3$ for this example.

Start with the manufactured solution $u(x) = x_1 + x_2$ and $- \Delta u = 0$.

In [26]:
u(x) = x[1] + x[2]
f(x) = 0

f (generic function with 1 method)

Give gradient of $u$ the exact value of the gradient of $u$.

In [4]:
∇u(x) = VectorValue(1,1)

∇u (generic function with 1 method)

Tell GridAp that the gradient of $u$ is the specified value.

In [29]:
import Gridap: ∇
∇(::typeof(u)) = ∇u

gradient (generic function with 34 methods)

In [31]:
∇(u) === ∇u

true

Set up the domain for the problem in 2D and give a quadrilateral partition to it. Set up GridAPs internal model.

In [7]:
domain = (0,1,0,1)
partition = (4,4)
model = CartesianDiscreteModel(domain,partition)

CartesianDiscreteModel()

Set up domain and model for the 3D problem.

In [8]:
domain3d = (0,1,0,1,0,1)
partition3d = (4,4,4)
model3d = CartesianDiscreteModel(domain3d,partition3d)

CartesianDiscreteModel()

In [9]:
writevtk(model,"model")

3-element Vector{Vector{String}}:
 ["model_0.vtu"]
 ["model_1.vtu"]
 ["model_2.vtu"]

Set up the function spaces, first order lagrangian spaces, then set up the test and trial spaces.

Next set the integration triangulation of the domain and the measure size of areas of domain partitions.

Integrate these to set up the linear system.

Solve the linear system.

In [10]:
order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
V0 = TestFESpace(model,reffe,conformity=:H1,dirichlet_tags="boundary")
U = TrialFESpace(V0,u)

degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

a(u,v) = ∫( ∇(v)⊙∇(u) )*dΩ
b(v) = ∫( v*f )*dΩ

op = AffineFEOperator(a,b,U,V0)

uh = solve(op)

SingleFieldFEFunction():
 num_cells: 16
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 280879764868874896

Set function to calculate the area

In [32]:
e = u - uh

OperationCellField():
 num_cells: 16
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 280879764868874896

In [12]:
writevtk(Ω,"error",cellfields=["e" => e])

(["error.vtu"],)

Calculate the $L2$ and $H1$ norms of the error.

In [13]:
el2 = sqrt(sum( ∫( e*e )*dΩ ))
eh1 = sqrt(sum( ∫( e*e + ∇(e)⋅∇(e) )*dΩ ))

1.0241643448331112e-15

In [14]:
tol = 1.e-10
@assert el2 < tol
@assert eh1 < tol

Set up the problem with a polynomial solution of order $p$, 2 in this case.

In [34]:
p = 3
u(x) = x[1]^p+x[2]^p
∇u(x) = VectorValue(p*x[1]^(p-1),p*x[2]^(p-1))
f(x) = -p*(p-1)*(x[1]^(p-2)+x[2]^(p-2))

f (generic function with 1 method)

Same as before

In [16]:
∇(::typeof(u)) = ∇u
b(v) = ∫( v*f )*dΩ

b (generic function with 1 method)

Run problem with the domain split into $n \times n$ boxes and order $k$ polynomials on the domain

In [35]:
function run(n,k)

  domain = (0,1,0,1)
  partition = (n,n)
  model = CartesianDiscreteModel(domain,partition)

  reffe = ReferenceFE(lagrangian,Float64,k)
  V0 = TestFESpace(model,reffe,conformity=:H1,dirichlet_tags="boundary")
  U = TrialFESpace(V0,u)

  degree = 2*p
  Ω = Triangulation(model)
  dΩ = Measure(Ω,degree)

  a(u,v) = ∫( ∇(v)⊙∇(u) )*dΩ
  b(v) = ∫( v*f )*dΩ

  op = AffineFEOperator(a,b,U,V0)

  uh = solve(op)

  e = u - uh

  el2 = sqrt(sum( ∫( e*e )*dΩ ))
  eh1 = sqrt(sum( ∫( e*e + ∇(e)⋅∇(e) )*dΩ ))

  (el2, eh1)

end

run (generic function with 1 method)

Function for convergence test, give a set of $n$s for the number of quads in the domain partition, and $k$ the order of the polynomial spaces.

In [18]:
function conv_test(ns,k)

  el2s = Float64[]
  eh1s = Float64[]
  hs = Float64[]

  for n in ns

    el2, eh1 = run(n,k)
    h = 1.0/n

    push!(el2s,el2)
    push!(eh1s,eh1)
    push!(hs,h)

  end

  (el2s, eh1s, hs)

end

conv_test (generic function with 1 method)

Get the errors and mesh parameter for the partitions.

In [19]:
el2s1, eh1s1, hs = conv_test([8,16,32,64,128],1);
el2s2, eh1s2, hs = conv_test([8,16,32,64,128],2);
nothing #hide

Get slope of graph for linear regression fit of errors against mesh parameter.

In [20]:
function slope(hs,errors)
  x = log10.(hs)
  y = log10.(errors)
  linreg = hcat(fill!(similar(x), 1), x) \ y
  linreg[2]
end

slope (generic function with 1 method)

Print out slopes

In [21]:
slope(hs,el2s1)
slope(hs,el2s2)

3.0000000002129363

In [22]:
slope(hs,eh1s1)slope(hs,eh1s1)
slope(hs,eh1s2)
slope(hs,eh1s2)

2.0000597414736574